## Method 1

#### What happens in this method:

* I’ll use **Hugging Face’s Sentence Transformer model** to convert both the user’s query and each product description into numerical embeddings locally (no API required).
* I’ll then compute **cosine similarity** using `sklearn.metrics.pairwise` to measure how semantically close each product is to the given query.
* The **`timeit`** library will be used to measure the time taken for embedding generation and similarity computation, helping assess efficiency.
* Finally, I’ll rank and display the top matching products based on their similarity scores to identify which ones best fit the user’s vibe.

In [1]:
def get_data():
    fashion_data = {
        "Boho Dress": {
            "desc": "A flowy, earthy-toned dress made from lightweight cotton fabric, perfect for outdoor festivals or beach walks. Its loose fit and floral embroidery reflect a free-spirited, nature-inspired aesthetic.",
            "vibes": ["boho", "cozy", "free-spirited"]
        },
        "Street Hoodie": {
            "desc": "An oversized streetwear hoodie featuring graffiti prints and bold typography. Ideal for casual city outings or skate park sessions, it embodies an energetic and urban vibe with a youthful edge.",
            "vibes": ["urban", "energetic", "casual"]
        },
        "Minimalist Blazer": {
            "desc": "A structured blazer with clean lines and a modern silhouette, designed for professionals who appreciate simplicity and elegance. Its neutral tones and tailored fit exude confidence and minimalism.",
            "vibes": ["minimal", "formal", "modern"]
        },
        "Denim Jacket": {
            "desc": "A rugged blue denim jacket with faded wash and metal buttons. This timeless piece adds a vintage cool factor and pairs effortlessly with both casual tees and stylish dresses.",
            "vibes": ["vintage", "casual", "cool"]
        },
        "Floral Maxi Dress": {
            "desc": "A soft chiffon maxi dress with delicate floral prints, ideal for garden parties or summer dates. The pastel hues and flowing fabric create a romantic, bohemian mood.",
            "vibes": ["feminine", "romantic", "boho"]
        },
        "Athletic Joggers": {
            "desc": "Slim-fit athletic joggers made from breathable stretch fabric for maximum comfort. Whether for gym sessions or urban streetwear looks, they combine performance and everyday versatility.",
            "vibes": ["sporty", "urban", "active"]
        },
        "Leather Biker Jacket": {
            "desc": "A premium black leather biker jacket with silver zippers and quilted shoulders. It adds a rebellious charm and chic boldness, making it a statement piece for any wardrobe.",
            "vibes": ["edgy", "chic", "rock"]
        },
        "Cozy Knit Sweater": {
            "desc": "A warm, chunky knit sweater crafted from soft wool-blend yarn. Designed for cold winter evenings, it wraps you in comfort and homeliness while maintaining a relaxed charm.",
            "vibes": ["cozy", "soft", "homey"]
        },
        "Silk Evening Gown": {
            "desc": "A luxurious silk gown with a flowing silhouette and subtle shimmer. Perfect for red-carpet events or elegant dinners, it radiates sophistication and timeless glamour.",
            "vibes": ["luxury", "classy", "glamorous"]
        },
        "Cargo Pants": {
            "desc": "Durable cargo pants featuring multiple utility pockets and adjustable straps. Their relaxed fit and earthy tones give off an adventurous, practical streetwear vibe.",
            "vibes": ["streetwear", "practical", "adventure"]
        },
        "Graphic Tee": {
            "desc": "A soft cotton t-shirt featuring vibrant abstract artwork and modern street graphics. A statement piece that captures creativity and urban self-expression.",
            "vibes": ["casual", "youthful", "artistic"]
        },
        "Linen Shirt": {
            "desc": "A breathable, lightweight linen shirt ideal for tropical vacations or relaxed weekends. Its natural texture and loose design make it a staple for minimal, effortless style.",
            "vibes": ["relaxed", "vacation", "minimal"]
        },
        "Plaid Flannel Shirt": {
            "desc": "A cozy flannel shirt with traditional plaid patterns and soft brushed fabric. Suitable for camping trips or casual coffee dates, it brings warmth and rustic charm.",
            "vibes": ["casual", "country", "warm"]
        },
        "Sequin Party Dress": {
            "desc": "A dazzling sequin-covered mini dress designed for nightlife and celebrations. It catches the light with every move, making it the centerpiece of any party look.",
            "vibes": ["party", "glamorous", "fun"]
        },
        "Turtleneck Sweater": {
            "desc": "A sleek ribbed turtleneck sweater that balances comfort with sophistication. Ideal for layering in fall, it gives off a smart minimalist vibe for both work and casual wear.",
            "vibes": ["minimal", "cozy", "smart"]
        },
        "Ripped Jeans": {
            "desc": "Distressed denim jeans featuring strategic rips and faded patches for a bold, rebellious appeal. A streetwear essential that pairs effortlessly with graphic tees or leather jackets.",
            "vibes": ["edgy", "urban", "cool"]
        },
        "Trench Coat": {
            "desc": "A long beige trench coat with a classic double-breasted design and belted waist. A timeless fashion piece that exudes elegance, confidence, and sophistication.",
            "vibes": ["classic", "formal", "elegant"]
        },
        "Summer Shorts": {
            "desc": "Lightweight cotton shorts with a relaxed fit, perfect for beach outings and summer walks. Their simplicity and comfort make them an easy choice for casual, sunny days.",
            "vibes": ["casual", "beach", "relaxed"]
        },
        "High Heels": {
            "desc": "Elegant stiletto heels with a sleek silhouette and glossy finish. Designed to elevate any outfit, they symbolize power, femininity, and high-end fashion.",
            "vibes": ["chic", "formal", "glamorous"]
        },
        "Wool Overcoat": {
            "desc": "A long wool overcoat tailored with precision, offering warmth and sophistication during chilly evenings. It’s a timeless staple that blends elegance with comfort.",
            "vibes": ["elegant", "cozy", "timeless"]
        }
    }
    return fashion_data

In [ ]:
from sentence_transformers import SentenceTransformer
from timeit import default_timer as timer
import pandas as pd

# Initialize model
model = SentenceTransformer("all-MiniLM-L6-v2")

def generate_description_embeddings(model):
    print("[INFO] Fetching fashion data...")
    fashion_data = get_data()

    print("[INFO] Creating DataFrame from product data...")
    fashion_df = pd.DataFrame([
        {"name": name, "desc": details["desc"], "vibes": details["vibes"]}
        for name, details in fashion_data.items()
    ])
    print(f"[INFO] Loaded {len(fashion_df)} products into DataFrame.\n")

    start = timer()

    # Encode all product descriptions
    embeddings = model.encode(fashion_df["desc"].tolist())

    # Store embeddings in the DataFrame
    fashion_df["embedding"] = list(embeddings)

    end = timer()
    execution_time = end - start

    print(f"[INFO] Embeddings generated successfully for {len(fashion_df)} descriptions.")
    print(f"[TIME] Embedding generation completed in {execution_time:.4f} seconds.\n")
    return fashion_df, execution_time

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from timeit import default_timer as timer

def vibe_matcher(query, model, top_k=3):
    fashion_df, execution_time = generate_description_embeddings(model)
    print("[INFO] Embedding generation complete. Proceeding with similarity computation.\n")
    start = timer()

    # Encode user query
    print(f"[INFO] Encoding {query}")
    encoded_query = model.encode(query)
    print(f"[INFO] Query encoded. Vector dimension: {len(encoded_query)}\n")

    # Convert stored embeddings to NumPy array
    embeddings = np.array(fashion_df["embedding"].to_list())

    # Compute cosine similarity
    print("[INFO] Computing cosine similarity between query and product vectors...")
    similarities = cosine_similarity([encoded_query], embeddings)[0]
    print("[INFO] Similarity computation completed.\n")

    # Create a new DataFrame for similarity results
    results_df = pd.DataFrame({
        "name": fashion_df["name"],
        "desc": fashion_df["desc"],
        "similarity_score": similarities
    }).sort_values(by="similarity_score", ascending=False).reset_index(drop=True)

    end = timer()
    execution_time = end - start

    # Output the latency
    print(f"Query processed in {execution_time:.4f} seconds.")
    print(f"[TIME] Query processing completed in {execution_time:.4f} seconds.\n")

    # Handle low-similarity fallback
    if results_df["similarity_score"].max() < 0.5:
        print("Result = No strong vibe match found. Try a different style description!")
        return None, execution_time
    else:
        return results_df.head(top_k), execution_time

## Usage and Logs

In [ ]:
vibes = [
    "cozy winter comfort",
    "romantic summer outfit",
    "edgy streetwear vibe",
    "beach vacation style",
    "glamorous party night",
    "classic formal elegance",
    "artistic youthful expression",
    "relaxed weekend mood"
]

In [ ]:
def run_pipeline(query, model, top_k=3):
    print("=" * 80)
    print(f"Running Vibe Matcher Pipeline for Query: '{query}'")
    print("=" * 80)

    results_df, execution_time = vibe_matcher(query, model, top_k)

    print("[INFO] Pipeline executed successfully.")
    print(f"[TIME] End-to-end execution time: {execution_time:.4f} seconds.\n")
    print("=" * 80)

    return results_df, execution_time

query = vibes[2]
results_df, execution_time = run_pipeline(query, model)
print(results_df)

Running Vibe Matcher Pipeline for Query: 'edgy streetwear vibe'
[INFO] Fetching fashion data...
[INFO] Creating DataFrame from product data...
[INFO] Loaded 20 products into DataFrame.

[INFO] Embeddings generated successfully for 20 descriptions.
[TIME] Embedding generation completed in 0.0127 seconds.

[INFO] Embedding generation complete. Proceeding with similarity computation.

[INFO] Encoding edgy streetwear vibe
[INFO] Query encoded. Vector dimension: 384

[INFO] Computing cosine similarity between query and product vectors...
[INFO] Similarity computation completed.

Query processed in 0.0160 seconds.
[TIME] Query processing completed in 0.0160 seconds.

[INFO] Pipeline executed successfully.
[TIME] End-to-end execution time: 0.0160 seconds.

            name                                               desc  \
0  Street Hoodie  An oversized streetwear hoodie featuring graff...   
1   Ripped Jeans  Distressed denim jeans featuring strategic rip...   
2    Cargo Pants  Durable c